In [55]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter

In [56]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [59]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender 
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [60]:
import optuna as op

In [61]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

print(f"Train size: {URM_train.getnnz()}, Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

Train size: 1274929, Val size: 224987, Test size: 264691


In [62]:
evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[100])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[100])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 527 ( 1.5%) Users that have less than 1 test interactions


In [63]:
def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

In [64]:
# We pass as paramether the recommender class

def evaluate_algorithm(URM_test, recommender_object, at=10):
    
    #cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, cutoff=at)
            num_eval+=1

            #cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    #cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    return MAP, cumulative_recall


# SLIM 

In [65]:
SLIM_ElasticRecall_train = SLIMElasticNetRecommender(URM_train)
#SLIM_ElasticRecall_eval.fit(topK= 296, l1_ratio=0.10442943478590742, alpha=0.00035921608889258387)
SLIM_ElasticRecall_train.load_model("result_experiments/SLIM/", "SLIM_ElasticRecall_eval")

SLIM_ElasticRecall_train_val = SLIMElasticNetRecommender(URM_train+URM_val)
SLIM_ElasticRecall_train_val.load_model("result_experiments/SLIM/", "SLIM_ElasticRecall_train_val_eval")

#SLIM_ElasticRecall_train_val_eval.fit(topK= 296, l1_ratio=0.10442943478590742, alpha=0.00035921608889258387)
#SLIM_ElasticRecall_train_val_eval.load_model('result_experiments/SLIM/', 'slim_307_train_val_ens')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/SLIM_ElasticRecall_eval'
SLIMElasticNetRecommender: Loading complete
SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/SLIM_ElasticRecall_train_val_eval'
SLIMElasticNetRecommender: Loading complete


In [66]:
slim_recall_train_res, _ = evaluator_val.evaluateRecommender(SLIM_ElasticRecall_train)
slim_recall_train_val_res, _ = evaluator_test.evaluateRecommender(SLIM_ElasticRecall_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 16.22 sec. Users per second: 2141
EvaluatorHoldout: Processed 35209 (100.0%) in 16.82 sec. Users per second: 2093


In [67]:
slim_recall_train_res["MAP"], slim_recall_train_res["RECALL"], slim_recall_train_val_res["MAP"], slim_recall_train_val_res["RECALL"]

(cutoff
 100    0.004474
 Name: MAP, dtype: object,
 cutoff
 100    0.319568
 Name: RECALL, dtype: object,
 cutoff
 100    0.007051
 Name: MAP, dtype: object,
 cutoff
 100    0.356014
 Name: RECALL, dtype: object)

# RP3beta

In [68]:
RP3BetaRecall_train = RP3betaRecommender(URM_train)
RP3BetaRecall_train.load_model("result_experiments/RP3beta/", "RP3BetaRecall_eval")

RP3BetaRecall_train_val = RP3betaRecommender(URM_train+URM_val)
#RP3BetaRecall_train_val_eval.fit(alpha=0.205870, beta=0.243986, topK=17, min_rating=0.0, implicit=False, normalize_similarity=True)
RP3BetaRecall_train_val.load_model("result_experiments/RP3beta/", "RP3BetaRecall_train_val_eval")

RP3betaRecommender: Loading model from file 'result_experiments/RP3beta/RP3BetaRecall_eval'
RP3betaRecommender: Loading complete
RP3betaRecommender: Loading model from file 'result_experiments/RP3beta/RP3BetaRecall_train_val_eval'
RP3betaRecommender: Loading complete


In [69]:
rp3_beta_recall_train_res, _ = evaluator_val.evaluateRecommender(RP3BetaRecall_train)
rp3_beta_recall_train_val_res, _ = evaluator_test.evaluateRecommender(RP3BetaRecall_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 11.40 sec. Users per second: 3046
EvaluatorHoldout: Processed 35209 (100.0%) in 11.58 sec. Users per second: 3041


In [70]:
rp3_beta_recall_train_res["MAP"], rp3_beta_recall_train_res["RECALL"], rp3_beta_recall_train_val_res["MAP"], rp3_beta_recall_train_val_res["RECALL"]

(cutoff
 100    0.00412
 Name: MAP, dtype: object,
 cutoff
 100    0.298366
 Name: RECALL, dtype: object,
 cutoff
 100    0.006317
 Name: MAP, dtype: object,
 cutoff
 100    0.330276
 Name: RECALL, dtype: object)

# KNN

In [71]:
KNNRecall_train = ItemKNNCFRecommender(URM_train)
KNNRecall_train.load_model("result_experiments/ItemKNNCFRecommender/", "KNNRecall_eval")

KNNRecall_train_val = ItemKNNCFRecommender(URM_train+URM_val)
KNNRecall_train_val.load_model("result_experiments/ItemKNNCFRecommender/", "KNNRecall_train_val_eval")

ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/KNNRecall_eval'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/KNNRecall_train_val_eval'
ItemKNNCFRecommender: Loading complete


In [72]:
knn_recall_train_res, _ = evaluator_val.evaluateRecommender(KNNRecall_train)
knn_recall_train_val_res, _ = evaluator_test.evaluateRecommender(KNNRecall_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 11.26 sec. Users per second: 3083
EvaluatorHoldout: Processed 35209 (100.0%) in 11.63 sec. Users per second: 3027


In [73]:
knn_recall_train_res["MAP"], knn_recall_train_res["RECALL"], knn_recall_train_val_res["MAP"], knn_recall_train_val_res["RECALL"]

(cutoff
 100    0.00403
 Name: MAP, dtype: object,
 cutoff
 100    0.257609
 Name: RECALL, dtype: object,
 cutoff
 100    0.006211
 Name: MAP, dtype: object,
 cutoff
 100    0.29597
 Name: RECALL, dtype: object)

# Current best Generator

In [74]:
results_df = pd.read_csv('result_experiments/ensemble_SLIM_RP3_KNNCFR/recall_3_01_2025.csv')

In [76]:
results_df.sort_values("RECALL_test", ascending=False)

,x_0,x_1,x_2,iteration,RECALL_val,SLIM_w,RP3b_w,ItemKNN_w,RECALL_test
46,0.306797,0.934627,0.889287,46,0.320836,0.864661,0.049475,0.085865,0.357465
21,0.286711,0.842840,0.847009,21,0.321111,0.787542,0.107784,0.104674,0.357425
36,0.063830,0.686867,0.729638,36,0.321044,0.799317,0.109116,0.091567,0.357420
70,0.050319,0.698360,0.606554,70,0.321052,0.776792,0.093292,0.129916,0.357274
19,0.290366,0.827032,0.800658,19,0.321088,0.749986,0.115179,0.134835,0.357229
...,...,...,...,...,...,...,...,...,...
37,0.338804,0.018184,0.854234,37,0.308291,0.206272,0.763702,0.030026,0.341955
0,0.550115,0.032974,0.758292,0,0.307364,0.139426,0.796023,0.064551,0.340988
86,0.983071,0.706092,0.711958,86,0.306289,0.024224,0.493757,0.482019,0.339976
2,0.925510,0.832883,0.367881,2,0.303312,0.061423,0.145098,0.793479,0.336458


In [89]:
new_similarityRecall_train = 0.7747783599151299 * SLIM_ElasticRecall_train.W_sparse + 0.10970438576060675 * RP3BetaRecall_train.W_sparse + 0.1155172543242634 * KNNRecall_train.W_sparse
candidate_gen_train = ItemKNNCustomSimilarityRecommender(URM_train)
candidate_gen_train.fit(new_similarityRecall_train)

In [90]:
new_similarityRecall_train_val = 0.7747783599151299 * SLIM_ElasticRecall_train_val.W_sparse + 0.10970438576060675 * RP3BetaRecall_train_val.W_sparse + 0.1155172543242634 * KNNRecall_train_val.W_sparse
candidate_gen_train_val = ItemKNNCustomSimilarityRecommender(URM_train+URM_val)
candidate_gen_train_val.fit(new_similarityRecall_train_val)

In [91]:
gen_recall_train_res, _ = evaluator_val.evaluateRecommender(candidate_gen_train)
gen_recall_train_val_res, _ = evaluator_test.evaluateRecommender(candidate_gen_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 15.39 sec. Users per second: 2256
EvaluatorHoldout: Processed 35209 (100.0%) in 16.41 sec. Users per second: 2146


In [92]:
gen_recall_train_res["MAP"], gen_recall_train_res["RECALL"], gen_recall_train_val_res["MAP"], gen_recall_train_val_res["RECALL"]

(cutoff
 100    0.004534
 Name: MAP, dtype: object,
 cutoff
 100    0.321244
 Name: RECALL, dtype: object,
 cutoff
 100    0.00711
 Name: MAP, dtype: object,
 cutoff
 100    0.357531
 Name: RECALL, dtype: object)

# Optuna

In [81]:
results_dict={
    'Iteration':[],
    'SLIM_w':[],
    'RP3b_w':[],
    'ItemKNN_w':[],
    'RECALL_val':[],
    'RECALL_test':[],
}

In [82]:
new_best_on_test={
    'SLIM_w':0,
    'RP3b_w':0,
    'ItemKNN_w':0,
    'RECALL_test':0
}

In [83]:
improving_treshold=0.70 

In [84]:
def objective_function_weight_hybrid(trial):
    cutoff = 100
    n = 3  # количество моделей
    model_keys = ['SLIM_w', 'RP3b_w', 'ItemKNN_w']
    
    # Генерация весов через логарифм и нормализацию
    x = -np.log([trial.suggest_float(f"x_{i}", 0, 1) for i in range(n)])
    weights = x / x.sum()
    
    # Логируем веса и сохраняем их в trial
    print(f"__________Iteration {trial.number}______________")
    for i, key in enumerate(model_keys):
        trial.set_user_attr(key, weights[i])  # Сохранение веса в user_attrs
        results_dict[key].append(weights[i])
        print(f"{key}: {weights[i]}")
    
    # Сохраняем номер итерации
    results_dict['Iteration'].append(trial.number)
    
    # Сбор гибридной матрицы
    new_similarity_train = (
        weights[0] * SLIM_ElasticRecall_train.W_sparse +
        weights[1] * RP3BetaRecall_train.W_sparse +
        weights[2] * KNNRecall_train.W_sparse
    )
    
    # Обучение модели на train данных
    recommender_object_train = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_object_train.fit(new_similarity_train)
    res_on_val, _ = evaluator_val.evaluateRecommender(recommender_object_train)
    recall_val = res_on_val['RECALL'][cutoff]
    results_dict['RECALL_val'].append(recall_val)
    
    # Проверка улучшений
    if recall_val >= best_on_val * improving_treshold:
        # Пересчёт для объединённого train+val
        new_similarity_train_val = (
            weights[0] * SLIM_ElasticRecall_train_val.W_sparse +
            weights[1] * RP3BetaRecall_train_val.W_sparse +
            weights[2] * KNNRecall_train_val.W_sparse
        )
        
        recommender_object_train_val = ItemKNNCustomSimilarityRecommender(URM_train + URM_val)
        recommender_object_train_val.fit(new_similarity_train_val)
        res_on_test, _ = evaluator_test.evaluateRecommender(recommender_object_train_val)
        recall_test = res_on_test['RECALL'][cutoff]
        results_dict['RECALL_test'].append(recall_test)
        
        # Проверка улучшений на тесте
        if recall_test > new_best_on_test['RECALL_test']:
            new_best_on_test.update({
                'SLIM_w': weights[0],
                'RP3b_w': weights[1],
                'ItemKNN_w': weights[2],
                'RECALL_test': recall_test
            })
            print(f"New best on test found! Score is {recall_test}")
            print(f"weights: {new_best_on_test}")
    else:
        results_dict['RECALL_test'].append(0.0)
        recall_test = 0.0

    # Сохраняем значения RECALL_test в user_attrs
    trial.set_user_attr("RECALL_test", recall_test)
    
    # Возврат значения для оптимизации
    return recall_val

In [85]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL_val"] = optuna_trial.value  # RECALL с валидации
        
        # Добавляем сохранённые веса и метрики из user_attrs
        trial_result["SLIM_w"] = optuna_trial.user_attrs.get("SLIM_w")
        trial_result["RP3b_w"] = optuna_trial.user_attrs.get("RP3b_w")
        trial_result["ItemKNN_w"] = optuna_trial.user_attrs.get("ItemKNN_w")
        trial_result["RECALL_test"] = optuna_trial.user_attrs.get("RECALL_test", 0.0)
        
        self.results.append(trial_result)
        
        # Сохраняем результаты в CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)

In [86]:
gen_save_res = SaveResults('result_experiments/ensemble_SLIM_RP3_KNNCFR/recall2_3_01_2025.csv')

In [87]:
study = op.create_study(direction="maximize")
study.optimize(objective_function_weight_hybrid,
                      callbacks=[gen_save_res],
                      n_trials = 300)

[I 2025-01-03 16:58:39,002] A new study created in memory with name: no-name-8215448c-4d88-445b-9354-6b00e6e22fb3


__________Iteration 0______________
SLIM_w: 0.047247060577294095
RP3b_w: 0.8210654621703851
ItemKNN_w: 0.1316874772523207
EvaluatorHoldout: Processed 34728 (100.0%) in 16.00 sec. Users per second: 2171
EvaluatorHoldout: Processed 35209 (100.0%) in 16.78 sec. Users per second: 2098


[I 2025-01-03 16:59:11,901] Trial 0 finished with value: 0.3057056160316105 and parameters: {'x_0': 0.9737486712745907, 'x_1': 0.6298372892021245, 'x_2': 0.9285365928939034}. Best is trial 0 with value: 0.3057056160316105.


New best on test found! Score is 0.3388074905737232
weights: {'SLIM_w': 0.047247060577294095, 'RP3b_w': 0.8210654621703851, 'ItemKNN_w': 0.1316874772523207, 'RECALL_test': 0.3388074905737232}
__________Iteration 1______________
SLIM_w: 0.2641317872416238
RP3b_w: 0.5198540053960069
ItemKNN_w: 0.2160142073623694
EvaluatorHoldout: Processed 34728 (100.0%) in 15.94 sec. Users per second: 2178
EvaluatorHoldout: Processed 35209 (100.0%) in 17.11 sec. Users per second: 2057


[I 2025-01-03 16:59:45,052] Trial 1 finished with value: 0.311064006759822 and parameters: {'x_0': 0.5611486495367921, 'x_1': 0.3207338780709533, 'x_2': 0.6234319487206639}. Best is trial 1 with value: 0.311064006759822.


...

[I 2025-01-03 19:42:09,145] Trial 297 finished with value: 0.3210436395865516 and parameters: {'x_0': 0.1538898578750174, 'x_1': 0.8227827498311158, 'x_2': 0.9675656520948394}. Best is trial 260 with value: 0.32143923701120636.


__________Iteration 298______________
SLIM_w: 0.856362041353412
RP3b_w: 0.1435308838114464
ItemKNN_w: 0.00010707483514171224
EvaluatorHoldout: Processed 34728 (100.0%) in 15.28 sec. Users per second: 2272
EvaluatorHoldout: Processed 35209 (100.0%) in 16.33 sec. Users per second: 2156


[I 2025-01-03 19:42:40,857] Trial 298 finished with value: 0.3206803287379547 and parameters: {'x_0': 0.22238574297181915, 'x_1': 0.7772694743028684, 'x_2': 0.9998120480096068}. Best is trial 260 with value: 0.32143923701120636.


__________Iteration 299______________
SLIM_w: 0.8739561463898916
RP3b_w: 0.09600800153875694
ItemKNN_w: 0.03003585207135144
EvaluatorHoldout: Processed 34728 (100.0%) in 15.23 sec. Users per second: 2281
EvaluatorHoldout: Processed 35209 (100.0%) in 16.36 sec. Users per second: 2152


[I 2025-01-03 19:43:12,542] Trial 299 finished with value: 0.3212183815951151 and parameters: {'x_0': 0.12873705808904157, 'x_1': 0.7983566482156493, 'x_2': 0.9319713687352246}. Best is trial 260 with value: 0.32143923701120636.


In [88]:
new_best_on_test

{'SLIM_w': 0.7747783599151299,
 'RP3b_w': 0.10970438576060675,
 'ItemKNN_w': 0.1155172543242634,
 'RECALL_test': 0.3575305308110468}